# Active Learner sobre 20 News Groups

## Obtengo el dataset desde sklearn

In [ ]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
from core import ActiveLearner, Dataset, Oracle
from sklearn.naive_bayes import MultinomialNB
from querys import CertaintySelector, UncertaintySelector, RandomSelector, MinDiffSelector, EntropySelector

In [ ]:
def get_n_each_category(dataset, n):
    train = []
    for cat in range(len(dataset.target_names)):
        count = 0
        i = 0
        while count < n and i < len(dataset.target):
            if dataset.target[i] == cat:
                train.append(i)
                count += 1
            i += 1
    train.sort()
    return train

In [ ]:
def remove_from_dataset(dataset, i):
    del dataset.data[i]
    dataset.target = np.delete(dataset.target, i)
    dataset.filenames = np.delete(dataset.filenames, i)
    return dataset

In [ ]:
def remove_many_from_dataset(dataset, indices):
    for i, index in enumerate(indices):
        dataset = remove_from_dataset(dataset, index-i)
    return dataset

In [ ]:
def split_train_data(dataset, train_indices):
    train_data = []
    train_target = []
    for i in train_indices:
        train_data.append(dataset.data[i])
        train_target.append(dataset.target[i])
    dataset = remove_many_from_dataset(dataset, train_indices)
    return dataset, train_data, train_target

In [ ]:
def clean_dataset(dataset):
    chars = set("abcdefghijklmnopqrstuvwxyz")
    to_remove = []
    for i in range(len(dataset.data)):
        dataset.data[i] = dataset.data[i].strip()
        dataset.data[i] = dataset.data[i].lower()
        if len(dataset.data[i]) == 0:
            to_remove.append(i)
        if not any((c in chars) for c in dataset.data[i]):
            to_remove.append(i)
    
    return remove_many_from_dataset(dataset, to_remove)

In [ ]:
categories = [
    'alt.atheism',
    'comp.graphics',
#    'comp.os.ms-windows.misc',
#    'comp.sys.ibm.pc.hardware',
#    'comp.sys.mac.hardware',
#    'comp.windows.x',
#    'misc.forsale',
#    'rec.autos',
#    'rec.motorcycles',
#    'rec.sport.baseball',
#    'rec.sport.hockey',
#    'sci.crypt',
#    'sci.electronics',
#    'sci.med',
    'sci.space',
#    'soc.religion.christian',
#    'talk.politics.guns',
#    'talk.politics.mideast',
#    'talk.politics.misc',
    'talk.religion.misc',
]

In [ ]:
dataset = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))#, categories=categories)

In [ ]:
dataset = clean_dataset(dataset)

In [ ]:
train_indices = get_n_each_category(dataset, 25)
dataset, train_data, train_target = split_train_data(dataset, train_indices)

In [ ]:
test_dataset = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))#, categories=categories)
test_data = test_dataset.data
test_target = test_dataset.target

## Obtengo los features TF-IDF

In [ ]:
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(train_data)
X_unlabeled = vectorizer.transform(dataset.data)
X_test = vectorizer.transform(test_data)

## Instancio lo minimo necesario para el framework

In [ ]:
class NGDataset(Dataset):
    dataset = dataset
    
    def get_unlabeled_readable(self, i):
        #return self.dataset.data[i]
        return self.dataset.target[i]


class NewsGroupOracle(Oracle):
    target_names = dataset.target_names
    
    def ask(self, X_readable, recoms):
        return X_readable

In [ ]:
selectors = [UncertaintySelector, CertaintySelector, MinDiffSelector, EntropySelector, RandomSelector, RandomSelector, RandomSelector]

In [ ]:
selector_scores = []
for selector in selectors:
    y_train = np.array(train_target)
    ngdataset = NGDataset(X_train, y_train, X_unlabeled)

    model = MultinomialNB(alpha=.01)
    oracle = NewsGroupOracle()
    al = ActiveLearner(model, ngdataset, selector, oracle)
    scores = []

    al.fit()
    scores.append(al.model.score(X_test, test_target))

    for _ in tqdm(range(40)):
        selected = al.select(10)
        y = al.ask(selected)
        al.tag_elements(selected, y)
        al.fit()
        scores.append(al.model.score(X_test, test_target))
    
    selector_scores.append(scores)

In [ ]:
plt.figure(figsize=(20,10))
plt.title('Comparación de selectores')
plt.xlabel('Iteraciones')
plt.ylabel('Precision')
plt.plot(selector_scores[0])
plt.plot(selector_scores[1])
plt.plot(selector_scores[2])
plt.plot(selector_scores[3])
plt.plot(selector_scores[4])
plt.plot(selector_scores[5])
plt.plot(selector_scores[6])
plt.legend(['Uncertainty', 'Certainty', 'MinDiff', 'Entropy', 'Random', 'Random', 'Random'])
#plt.savefig('nose')
plt.show()